The NHS dm+d contains a flag which marks if something is unavalable. A related [notebook](https://gist.github.com/sebbacon/317938750ea79e6291ae2cac51a0d31b) uncovered reimbursement against a prouct which dm+d reports as unavailable for the last five years. Here we invesitigate reimbursement of unavailable products.

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
##ensure £ and pence are set right
pd.set_option('display.float_format', lambda x: '%.2f' % x)

There is a flag at vmp level relating to availability where all AMP(P) should be unavailable according to the documentation of dm+d

In [3]:
sql = '''
SELECT
  Date,
  TRIM(Principal_Supplier) AS supplier,
  bnf_name,
  presc.bnf_code,
  vmp.id,
  non_availdt,
  Items,
  actual_Cost
FROM
  ebmdatalab.hscic.prescribing_2019_04 AS presc ##latest month
Inner JOIN
  ebmdatalab.dmd.vmp AS vmp
ON
  presc.bnf_code = vmp.bnf_code
JOIN
  ebmdatalab.alex.vendors AS software #this is where the up to date vendors table is held
ON
  software.ODS = presc.practice
  AND Date = presc.month
WHERE
  non_avail = 1 ###1 = Actual Products not Available
GROUP BY
  Date,
  supplier,
  bnf_name,
  presc.bnf_code,
  vmp.id,
  non_avail,
  non_availdt,
  Items,
  actual_Cost
'''



df_unavailable_vmp = bq.cached_read(sql, csv_path='amp_dmd_unavailable_vmp.csv')
df_unavailable_vmp.head(50)

,Date,supplier,bnf_name,bnf_code,id,non_availdt,Items,actual_Cost
0,2019-04-01,EMIS,K-Y Lubri Jelly,21140000005,34159511000001108,2019-02-01,1,2.18
1,2019-04-01,EMIS,K-Y Lubri Jelly,21140000005,34159511000001108,2019-02-01,1,8.67
2,2019-04-01,EMIS,Midazolam_OromucSoln 2.5mg/0.5mlPfos S/F,0408020W0AAAAAA,17020911000001109,2016-12-12,2,304.77
3,2019-04-01,TPP,Surg Suture W319 Ethilon Non-Absorb Ster Polya...,20110000240,7291811000001107,2019-02-01,15,20.65
4,2019-04-01,EMIS,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,4,23.65
5,2019-04-01,EMIS,K-Y Lubri Jelly,21140000005,34159511000001108,2019-02-01,1,4.34
6,2019-04-01,TPP,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,5,4.94
7,2019-04-01,EMIS,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,10,32.78
8,2019-04-01,EMIS,K-Y Lubri Jelly,21140000005,34159511000001108,2019-02-01,1,2.16
9,2019-04-01,TPP,SMA_PRO Gold Prem 2 Pdr,090900000BBLFA0,33665911000001107,2019-03-12,2,137.16


In [4]:
#monthly total
df_unavailable_vmp.sum()[['Items','actual_Cost']]

Items             40738
actual_Cost   764565.14
dtype: object

In [7]:
###lets look at individual products
df_unavailable_vmp.bnf_name.unique()

array(['K-Y Lubri Jelly', 'Midazolam_OromucSoln 2.5mg/0.5mlPfos S/F',
       'Surg Suture W319 Ethilon Non-Absorb Ster Polyam 6 Monof',
       'Nifedipine_Cap 10mg', 'SMA_PRO Gold Prem 2 Pdr',
       'U100 Syrg Sle Use 0.5ml + 12mm Needle-Ster Hypod Syrg',
       'Olive Oil_Ear Dps', 'Nifedipine_Cap 5mg',
       'Surg Suture W320 Ethilon Non-Absorb Ster Polyam 6 Monof',
       'SMA_Gold Prem 2 Catch-up For Pdr', 'SMA_PRO High Energy Liq',
       'U100 Syrg Sle Use 1ml + 12mm Needle-Ster Hypod Syrg',
       'Silver Nit Caustic_Applic 75%',
       'Midazolam_Oromuc Soln 10mg/2ml Pfos S/F',
       'SMA_PRO Gold Prem 2 Liq', 'Homeopathic Preparation - Proprietary',
       'Nutramigen 2 LIPIL_Pdr',
       'Allevyn Gentle 5cm x 5cm Wound Dress Soft Polymer',
       'Verapamil HCl_Cap 120mg M/R', 'SMA_PRO Anti-reflux Pdr',
       'Kliniderm fiber cmc 10cm x 10cm Wound Dress Protease Matrix',
       'Nedocromil Sod_Eye Dps Aq 2%', 'Acetic Acid_Ear Spy 2% 5ml',
       'Co-Danthrusate_Cap 50mg/6

In [14]:
df_unavailable_vmp.bnf_name.nunique()

202

In [19]:
##this is a list of products that look like they may be available
interesting = ['Nifedipine_Cap 10mg', 'Estradiol/Drospirenone_Tab 1mg/2mg', 'Deferiprone_Tab 500mg', 'Nifedipine_Cap 5mg', 'Olive Oil_Ear Dps', 'Colistimeth Sod_Neb Pdr 2mega u']

In [20]:
df_unavailable_vmp[df_unavailable_vmp.bnf_name.isin(interesting)]

,Date,supplier,bnf_name,bnf_code,id,non_availdt,Items,actual_Cost
4,2019-04-01,EMIS,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,4,23.65
6,2019-04-01,TPP,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,5,4.94
7,2019-04-01,EMIS,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,10,32.78
10,2019-04-01,TPP,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,2,7.08
12,2019-04-01,TPP,Olive Oil_Ear Dps,1201030H0AAAAAA,3393511000001105,2018-12-12,1,1.92
13,2019-04-01,TPP,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,3,12.14
15,2019-04-01,EMIS,Nifedipine_Cap 5mg,0206020R0AAAAAA,319222004,2019-02-18,1,1.65
17,2019-04-01,EMIS,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,5,13.56
22,2019-04-01,EMIS,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,3,20.38
25,2019-04-01,TPP,Nifedipine_Cap 5mg,0206020R0AAAAAA,319222004,2019-02-18,1,2.59


INvestigations cross referencing dm+browser, BNF and emc show that all of these idientified products are truly unavailable with this name and form. Some products have been discontinued e.g. nifedipine while others like Colistimeth Sod are possibly just cosmetic name or technical license changes

In [23]:
Homepathic but we 
homeo = ['Homeopathic Preparation - Proprietary']

In [24]:
df_unavailable_vmp[df_unavailable_vmp.bnf_name.isin(homeo)]

,Date,supplier,bnf_name,bnf_code,id,non_availdt,Items,actual_Cost
74,2019-04-01,TPP,Homeopathic Preparation - Proprietary,190203000BBAAA0,5049511000001109,2017-01-01,5,19.57
75,2019-04-01,TPP,Homeopathic Preparation - Proprietary,190203000BBAAA0,5131511000001104,2016-02-15,5,19.57
76,2019-04-01,TPP,Homeopathic Preparation - Proprietary,190203000BBAAA0,5131011000001107,2017-01-01,5,19.57
77,2019-04-01,TPP,Homeopathic Preparation - Proprietary,190203000BBAAA0,5132911000001100,2016-02-15,5,19.57
78,2019-04-01,TPP,Homeopathic Preparation - Proprietary,190203000BBAAA0,19899711000001101,2016-10-26,5,19.57
79,2019-04-01,TPP,Homeopathic Preparation - Proprietary,190203000BBAAA0,19899811000001109,2016-11-07,5,19.57
80,2019-04-01,TPP,Homeopathic Preparation - Proprietary,190203000BBAAA0,19899911000001104,2016-11-07,5,19.57
81,2019-04-01,TPP,Homeopathic Preparation - Proprietary,190203000BBAAA0,19900011000001101,2016-10-26,5,19.57
82,2019-04-01,TPP,Homeopathic Preparation - Proprietary,190203000BBAAA0,5124611000001101,2010-12-07,5,19.57
83,2019-04-01,TPP,Homeopathic Preparation - Proprietary,190203000BBAAA0,5125011000001107,2010-12-07,5,19.57


In [31]:
df_unavailable_vmp[df_unavailable_vmp.bnf_name.isin(homeo)].sum(numeric_only = True)

id            148816419220020166656.00
Items                         28644.00
actual_Cost                  523778.92
dtype: float64